In [ ]:
import psycopg2
import pandas as pd


In [ ]:
DATABASE_URL="postgres://u2bbja15c4igeb:pedd96ce18167a13cc8d49f93dc8db09b97790d992dfcf068d16b08f4d0edbd0b@c4c161t4pf58h3.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d421pu93ggcr06"


In [ ]:
def execute_sql(sql, return_all=False, commit=False):
    try:

        con = psycopg2.connect(DATABASE_URL, sslmode='require')
        cursor = con.cursor()
        cursor.execute(sql)

        if commit:
            con.commit()

        if return_all:
            out = cursor.fetchall()
            con.close()
            return out

        con.close()

    except psycopg2.OperationalError as e:
        try:
            print(e)
            con.close()
        except Exception:
            pass

# List all tables in the database
tables = execute_sql("""
    SELECT table_schema, table_name 
    FROM information_schema.tables 
    WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
    ORDER BY table_schema, table_name
""", return_all=True)

print("Available tables:")
for schema, table in tables:
    print(f"{schema}.{table}")

In [ ]:
# retreives 10 rows with all columns from the review table
data = execute_sql("SELECT * FROM reviews LIMIT 200", return_all=True)


In [ ]:
def tuples_to_dataframe(data_tuples_list):
    # Create list of dictionaries
    data_dicts = []
    
    for tup in data_tuples_list:
        data_dict = {
            'user_id': tup[0],
            'business_id': tup[1],
            'rating': tup[2],
            'review': tup[3],
            'date': tup[4]
        }
        data_dicts.append(data_dict)
    
    # Create DataFrame from the list of dictionaries
    df = pd.DataFrame(data_dicts)
    
    # Convert date string to datetime object (optional)
    df['date'] = pd.to_datetime(df['date'])
    
    return df


reviews = tuples_to_dataframe(data)

reviews.to_pickle('reviews.pkl')